## **0. Introduction**

This notebook follows after the "Logistic_Regression.iypnb" notebook. Aim of this notebook is to conduct the Support Vector Machine algorithm. 

## **1. Masking**

As in the Logistic Regression notebook, we will first load the masks, load the data and then apply the mask. I will not explain each step, because this has already been described in the aformentioned notebook.

In [1]:
import os

os.chdir("/home/jpauli/ds001506/sourcedata/sub-02/anat")
os.listdir()

['sub-02_mask_LH_FFA.nii.gz',
 'sub-02_mask_LH_hV4.nii.gz',
 'sub-02_mask_LH_HVC.nii.gz',
 'sub-02_mask_LH_LOC.nii.gz',
 'sub-02_mask_LH_PPA.nii.gz',
 'sub-02_mask_LH_V1d.nii.gz',
 'sub-02_mask_LH_V1v.nii.gz',
 'sub-02_mask_LH_V2d.nii.gz',
 'sub-02_mask_LH_V2v.nii.gz',
 'sub-02_mask_LH_V3d.nii.gz',
 'sub-02_mask_LH_V3v.nii.gz',
 'sub-02_mask_RH_FFA.nii.gz',
 'sub-02_mask_RH_hV4.nii.gz',
 'sub-02_mask_RH_HVC.nii.gz',
 'sub-02_mask_RH_LOC.nii.gz',
 'sub-02_mask_RH_PPA.nii.gz',
 'sub-02_mask_RH_V1d.nii.gz',
 'sub-02_mask_RH_V1v.nii.gz',
 'sub-02_mask_RH_V2d.nii.gz',
 'sub-02_mask_RH_V2v.nii.gz',
 'sub-02_mask_RH_V3d.nii.gz',
 'sub-02_mask_RH_V3v.nii.gz']

In [2]:
mask_img_path_sub1 = '/home/jpauli/ds001506/sourcedata/sub-01/anat'
mask_img_path_sub2 = '/home/jpauli/ds001506/sourcedata/sub-02/anat'

mask_img_sub1 = os.path.join(mask_img_path_sub1,'sub-01_mask_LH_LOC.nii.gz')
mask_img_sub2 = os.path.join(mask_img_path_sub2,'sub-02_mask_LH_LOC.nii.gz')


In [5]:
from nilearn.maskers import NiftiMasker
nifti_masker = NiftiMasker(mask_img=mask_img_sub1)
#nifti_masker2 = NiftiMasker(mask_img=mask_img_sub2)

X = []
Y = []
Sessions = []
category = 1
cat_index = 0
cat_index_boundary = 5
for session in ["1","2","3","4"]:
    os.chdir('/mnt/c/Users/janos/git/sessions_new/z_maps_{}_perrun'.format(session))
    for x in os.listdir():
        if x == 'nilearn_cache':
            continue
        else:
            X.append(nifti_masker.fit_transform(x))
            Y.append(category)
            Sessions.append(session)
            cat_index = cat_index + 1
            if cat_index == cat_index_boundary:
                category=category+1
                cat_index_boundary = cat_index_boundary+5
            if category == 27:
                category = 1

In [15]:
import os
import warnings
warnings.filterwarnings(action='once')

from nilearn import datasets
# We are fetching the data for subject 4
data_dir = os.path.join('..', 'data')
sub_no = 4
haxby_dataset = datasets.fetch_haxby(subjects=[sub_no], fetch_stimuli=True, data_dir=data_dir)
func_file = haxby_dataset.func[0]

# mask the data
from nilearn.input_data import NiftiMasker
mask_filename = haxby_dataset.mask_vt[0]
masker = NiftiMasker(mask_img=mask_filename, standardize=True, detrend=True)
X_ = masker.fit_transform(func_file)

# cognitive annotations
import pandas as pd
behavioral = pd.read_csv(haxby_dataset.session_target[0], delimiter=' ')
y = behavioral['labels']


Dataset created in ../data/haxby2001



 ...done. (1 seconds, 0 min)


 ...done. (1 seconds, 0 min)


Downloaded 329695232 of 329954386 bytes (99.9%,    0.3s remaining) ...done. (408 seconds, 6 min)
Extracting data from ../data/haxby2001/a0922102088d16a1a5cc2ae5c121d855/subj4-2010.01.14.tar.gz..... done.


Downloaded 9609216 of 10463676 bytes (91.8%,    0.6s remaining) ...done. (7 seconds, 0 min)
Extracting data from ../data/haxby2001/0a25a08cd199b76ff7f6fe9a2d8710e7/stimuli-2010.01.14.tar.gz..... done.
/home/jpauli/miniconda3/envs/neuro_ai/lib/python3.7/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)
/home/jpauli/miniconda3/envs/neuro_ai/lib/python3.7/site-packages/nilearn/image/resampling.py:453: UserWarning: The provided image has no sform in its header. Please check the provided file. Results may not be as expected.
  warnings.warn("The provided image has no sform in its header. "


In [7]:
import numpy as np
import pandas as pd

Y = np.array(Y)
Y_ses = np.array(Sessions)
df =  pd.DataFrame(np.concatenate(X))
X_ = df.to_numpy()




In [21]:
y.shape

(1452,)

In [22]:
X_.shape

(1452, 675)

## **1.0 Support Vector Machine**

In [10]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_new = SelectKBest(f_classif, k=10).fit_transform(X_, y)
X_new.shape

(520, 10)

In [23]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=42)

In [24]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X_train_s = scaler.fit_transform(X_train)

In [25]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
clf = svm.SVC(C = 0.01, kernel='linear') # Linear Kernel

clf.fit(X_train_s, y_train)

y_pred = clf.predict(scaler.transform(X_test))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bottle       0.79      0.58      0.67        26
         cat       0.86      0.90      0.88        20
       chair       0.83      0.77      0.80        13
        face       0.93      0.88      0.90        16
       house       0.97      0.97      0.97        34
        rest       0.85      0.91      0.88       123
    scissors       0.67      0.67      0.67        21
scrambledpix       0.83      0.68      0.75        22
        shoe       0.75      0.94      0.83        16

    accuracy                           0.85       291
   macro avg       0.83      0.81      0.82       291
weighted avg       0.85      0.85      0.84       291



In [14]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf','sigmoid']} 
  
grid = GridSearchCV(svm.SVC(), param_grid,cv = 10,refit = True, verbose = 3)
# fitting the model for grid search
grid.fit(X_train_s, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 2/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 3/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 4/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 5/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 6/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 7/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 8/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 9/10] END ....C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 10/10] END ...C=0.1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 1/10] END ......C=0.1, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 2/10] END ......C=0.1, gamma=1, kernel=p

[CV 3/10] END ....C=0.1, gamma=0.01, kernel=rbf;, score=0.000 total time=   0.1s
[CV 4/10] END ....C=0.1, gamma=0.01, kernel=rbf;, score=0.000 total time=   0.1s
[CV 5/10] END ....C=0.1, gamma=0.01, kernel=rbf;, score=0.024 total time=   0.1s
[CV 6/10] END ....C=0.1, gamma=0.01, kernel=rbf;, score=0.024 total time=   0.1s
[CV 7/10] END ....C=0.1, gamma=0.01, kernel=rbf;, score=0.024 total time=   0.1s
[CV 8/10] END ....C=0.1, gamma=0.01, kernel=rbf;, score=0.024 total time=   0.1s
[CV 9/10] END ....C=0.1, gamma=0.01, kernel=rbf;, score=0.000 total time=   0.1s
[CV 10/10] END ...C=0.1, gamma=0.01, kernel=rbf;, score=0.000 total time=   0.1s
[CV 1/10] END C=0.1, gamma=0.01, kernel=sigmoid;, score=0.048 total time=   0.1s
[CV 2/10] END C=0.1, gamma=0.01, kernel=sigmoid;, score=0.024 total time=   0.1s
[CV 3/10] END C=0.1, gamma=0.01, kernel=sigmoid;, score=0.000 total time=   0.1s
[CV 4/10] END C=0.1, gamma=0.01, kernel=sigmoid;, score=0.048 total time=   0.1s
[CV 5/10] END C=0.1, gamma=0

[CV 4/10] END ......C=1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 5/10] END ......C=1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 6/10] END ......C=1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 7/10] END ......C=1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 8/10] END ......C=1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 9/10] END ......C=1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 10/10] END .....C=1, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 1/10] END ........C=1, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 2/10] END ........C=1, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 3/10] END ........C=1, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 4/10] END ........C=1, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 5/10] END ........C=1, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 6/10] END ........C=1, g

[CV 7/10] END ......C=1, gamma=0.01, kernel=rbf;, score=0.024 total time=   0.1s
[CV 8/10] END ......C=1, gamma=0.01, kernel=rbf;, score=0.024 total time=   0.1s
[CV 9/10] END ......C=1, gamma=0.01, kernel=rbf;, score=0.000 total time=   0.1s
[CV 10/10] END .....C=1, gamma=0.01, kernel=rbf;, score=0.000 total time=   0.1s
[CV 1/10] END ..C=1, gamma=0.01, kernel=sigmoid;, score=0.024 total time=   0.1s
[CV 2/10] END ..C=1, gamma=0.01, kernel=sigmoid;, score=0.000 total time=   0.1s
[CV 3/10] END ..C=1, gamma=0.01, kernel=sigmoid;, score=0.024 total time=   0.1s
[CV 4/10] END ..C=1, gamma=0.01, kernel=sigmoid;, score=0.071 total time=   0.2s
[CV 5/10] END ..C=1, gamma=0.01, kernel=sigmoid;, score=0.024 total time=   0.2s
[CV 6/10] END ..C=1, gamma=0.01, kernel=sigmoid;, score=0.095 total time=   0.1s
[CV 7/10] END ..C=1, gamma=0.01, kernel=sigmoid;, score=0.073 total time=   0.1s
[CV 8/10] END ..C=1, gamma=0.01, kernel=sigmoid;, score=0.073 total time=   0.1s
[CV 9/10] END ..C=1, gamma=0

[CV 10/10] END ....C=10, gamma=1, kernel=linear;, score=0.000 total time=   0.1s
[CV 1/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 2/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 3/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 4/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 5/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 6/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 7/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 8/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 9/10] END .......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 10/10] END ......C=10, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 1/10] END ........C=10, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 2/10] END ........C=10, 

...

[CV 6/10] END ......C=100, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 7/10] END ......C=100, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 8/10] END ......C=100, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 9/10] END ......C=100, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 10/10] END .....C=100, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 1/10] END .......C=100, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 2/10] END .......C=100, gamma=1, kernel=rbf;, score=0.000 total time=   0.1s
[CV 3/10] END .......C=100, gamma=1, kernel=rbf;, score=0.024 total time=   0.2s
[CV 4/10] END .......C=100, gamma=1, kernel=rbf;, score=0.000 total time=   0.1s
[CV 5/10] END .......C=100, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 6/10] END .......C=100, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 7/10] END .......C=100, gamma=1, kernel=rbf;, score=0.049 total time=   0.1s
[CV 8/10] END .......C=100, 

[CV 9/10] END C=100, gamma=0.01, kernel=sigmoid;, score=0.122 total time=   0.1s
[CV 10/10] END C=100, gamma=0.01, kernel=sigmoid;, score=0.024 total time=   0.1s
[CV 1/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 2/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 3/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 4/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 5/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 6/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 7/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 8/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 9/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 10/10] END C=100, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 1/10] END ..C=100, gam

[CV 10/10] END ....C=1000, gamma=1, kernel=poly;, score=0.000 total time=   0.1s
[CV 1/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 2/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.000 total time=   0.1s
[CV 3/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 4/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.000 total time=   0.1s
[CV 5/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 6/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 7/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.049 total time=   0.1s
[CV 8/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.024 total time=   0.1s
[CV 9/10] END ......C=1000, gamma=1, kernel=rbf;, score=0.000 total time=   0.1s
[CV 10/10] END .....C=1000, gamma=1, kernel=rbf;, score=0.000 total time=   0.1s
[CV 1/10] END ..C=1000, gamma=1, kernel=sigmoid;, score=0.024 total time=   0.1s
[CV 2/10] END ..C=1000, gamm

[CV 2/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 3/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.2s
[CV 4/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 5/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 6/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 7/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 8/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 9/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 10/10] END C=1000, gamma=0.001, kernel=linear;, score=0.000 total time=   0.1s
[CV 1/10] END .C=1000, gamma=0.001, kernel=poly;, score=0.000 total time=   0.1s
[CV 2/10] END .C=1000, gamma=0.001, kernel=poly;, score=0.000 total time=   0.1s
[CV 3/10] END .C=1000, gamma=0.001, kernel=poly;, score=0.000 total time=   0.1s
[CV 4/10] END .C=1

GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=3)

In [ ]:
#print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
grid_predictions = grid.predict(X_test)
  
# print classification report
print(classification_report(y_test, grid_predictions))

In [ ]:
y_train

In [ ]:
grid.predict(X_train_s)

In [ ]:
grid.score(X_train_s,y_train)

In [ ]:
grid.score(X_test,y_test)

In [ ]:
Y_unique = np.unique(Y)
Y_unique

In [ ]:
np.random.shuffle(Y)
y_train = []
y_test = []


for category in Y:
        if len(y_test) == len(Y_unique):
            y_train.append(category)
        else:
            y_test.append(category)
            if y_test.count(category) > 1:
                y_test.remove(category)
                y_train.append(category)
                continue
            
                

In [ ]:
y_test = np.array(y_test)
y_train = np.array(y_train)

In [ ]:
X_train, X_test = train_test_split(X_, test_size=0.33, random_state=42)

In [ ]:
X_train_s_new = scaler.fit_transform(X_train)

In [ ]:
X_train.shape

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf','sigmoid']} 
  
grid = GridSearchCV(svm.SVC(), param_grid,cv = 2,refit = True, verbose = 3)
# fitting the model for grid search
grid.fit(X_train_s_new, y_train)

In [ ]:
grid.score(scaler.transform(X_test), y_test)

In [ ]:
grid.score(X_train_s_new, y_train)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA() #n_components = 10
pca.fit(X_)

In [ ]:
plt.plot(pca.explained_variance_ratio_)

In [ ]:
X_transform = pca.transform(X_)

In [ ]:
import matplotlib.pyplot as plt
corr =np.corrcoef(X_transform)
plt.matshow(corr, cmap='coolwarm')
plt.colorbar()
plt.xlabel('Samples')
plt.ylabel('Samples')

In [ ]:
fig, ax =plt.subplots(figsize=(12, 4), ncols=3, sharey=True, sharex=True)
ax[0].scatter(X_transform[:, 0], X_transform[:, 1])
ax[1].scatter(X_transform[:, 1], X_transform[:, 2])
ax[2].scatter(X_transform[:, 0], X_transform[:, 2])
plt.xlabel('PC 1')
plt.ylabel('PC 2')

In [ ]:
rand_X = np.random.normal(size=X_.shape)
pca_rand = PCA()
pca_rand.fit(rand_X)

In [ ]:
plt.plot(pca.explained_variance_ratio_[:10])
plt.plot(pca_rand.explained_variance_ratio_[:10])